In [0]:
dbutils.widgets.text("proc_date", "")
proc_date = dbutils.widgets.get("proc_date")

In [0]:
next_layer = "silver"

In [0]:
%run ../../../utils/common

In [0]:
dbutils.widgets.text("environment", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
# Dynamic select query to read the data for specific proc_date
dynamic_query = f""" CREATE OR REPLACE TEMP VIEW temp_pos_wcm_saleout AS SELECT *
FROM udp_wcm_pro.udp_wcm_bronze_pro.pos_wcm_saleout
WHERE ProcDate like '{str(proc_date).replace('-', '')}%'
"""

# Execute the query
spark.sql(dynamic_query)

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_silver_realtime.pos_wcm_saleout_bill_header
(
hash_id BIGINT,
receipt_no STRING,
pos_no STRING,
store_id STRING,
calday DATE,
caltime STRING, --BQ datatype for column caltime was TIME but in databricks we don't have it so kept string
member_card_no STRING,
vinid_csn STRING,
header_ref_01 STRING,
header_ref_02 STRING,
header_ref_03 STRING,
header_ref_04 STRING,
header_ref_05 STRING,
proc_date TIMESTAMP
) 
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'true',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'true'
)
""")

In [0]:
spark.sql(
    f"""
CREATE OR REPLACE TEMP VIEW view_pos_wcm_saleout_bill_header as
WITH dat AS (
    SELECT 
        LTRIM('0', main_tbl.ReceiptNo) AS receipt_no,
        LTRIM('0', main_tbl.PosNo) AS pos_no,
        LTRIM('0', main_tbl.StoreCode) AS store_id,
        TO_DATE(TO_TIMESTAMP(main_tbl.CalendarDay, 'MM/dd/yyyy HH:mm:ss')) AS calday,
        date_format(to_timestamp(substr(TranTime, 1, 6), 'HHmmss'),'HH:mm:ss') AS caltime,
        main_tbl.MemberCardNo AS member_card_no,
        main_tbl.VinidCsn AS vinid_csn,
        main_tbl.header_ref_01 AS header_ref_01,
        main_tbl.header_ref_02 AS header_ref_02,
        main_tbl.header_ref_03 AS header_ref_03,
        main_tbl.header_ref_04 AS header_ref_04,
        main_tbl.header_ref_05 AS header_ref_05,
        TO_TIMESTAMP(main_tbl.ProcDate, 'yyyyMMddHHmmss') AS proc_date,
        transline1.MemberPointsEarn as member_points_earn,
        transline1.MemberPointsRedeem as member_points_redeem
        FROM temp_pos_wcm_saleout AS main_tbl LATERAL VIEW EXPLODE(main_tbl.TransLine) AS transline1
)
SELECT 
    farm_fingerprint(CONCAT(
        IFNULL(CAST(receipt_no AS STRING), ""),
        IFNULL(CAST(pos_no AS STRING), ""),
        IFNULL(CAST(store_id AS STRING), "")
    )) AS hash_id,
    *
FROM dat
QUALIFY ROW_NUMBER() OVER (PARTITION BY receipt_no ORDER BY proc_date DESC) = 1     
"""
)

In [0]:
spark.sql(f"DELETE FROM {catalog_name}.udp_wcm_silver_realtime.pos_wcm_saleout_bill_header WHERE RECEIPT_NO IN (SELECT RECEIPT_NO FROM view_pos_wcm_saleout_bill_header)")

In [0]:
spark.sql(f"INSERT INTO {catalog_name}.udp_wcm_silver_realtime.pos_wcm_saleout_bill_header SELECT * FROM view_pos_wcm_saleout_bill_header")